Obtain financial data (at least three assets) of your own choice

• Strategy Implementation:

• Implement at least three strategies

• Use rolling window approach to obtain out-of-sample results

• Report final performance metrics (CAGR, annual volatility, Sharpe ratio, and max drawdown)

We picked the US sector ETFs ["XLB","XLE","XLF","XLI","XLK","XLP","XLU","XLV","XLY","XHB","XRT","XSD"], along with 2 popular US ETFs, QQQ and SPY to construct our portfolios. 

We tried 3 different strategies, max sharpe, minimum variance and target return portfolios (targeting a return of 7% annualized).

For all 3 strategies, we used a rolling window approach to construct the portfolios, with 5 years of data for "training" (calculating the mean return and covariance matrix) to create a portfolio that will be used for the next 1 month. 

We start our portfolio from January 2015 (so we need data from January 2010) to construct the portfolio, rebalancing every month.

We constrain our portfolio to be long only (weight to be from 0 to 1).

We do not include any trading costs, stop loss or rebalancing within the month.

For the Tangency Portfolio,

We use US 1 month rate (obtained from FRED via pandas datareader) as the risk free rate.

We note that despite creating an optimized portfolio that should achieve the max Sharpe using past 5 years of data, the actual portfolio created does not achieve higher Sharpe on a forward looking basis.

For the minimum variance portfolio,

The risk-free rate is not needed in the calculation.

The optimized portfolio does indeed have lower standard deviation on a forward looking basis.

The annualized return is notably lower, reflecting the concept that lower risk -> lower return.

The maximum drawdown is also higher than a portfolio created tageting 7% annualized returns.

For the target return portfolio,

We target a return of 7%, which is slightly below the long-term expected annualized return of SP500.

We note that for a single month in our period of analysis, our optimizer failed to return the optimized weights. Upon investigation, we found that the 5 year annualized mean return of all the ETFs are below 7%. This was because the month in question was March 2020, and the large drawdown in that month wiped out all gains over the past 5 years.

# 👉 <a id = "top">Table of Contents</a> 👈 

# [1. Functions](#p1)

# [2. Download data from yfinance](#p2)

# [3. Preparing data for portfolio optimization](#p3)

### [3a. Need to get Third Thursday Dates](#p3a)

### [3b. Need to get S&P to get market returns](#p3b)

### [3c. Creating inputs for MVO](#p3c)

# [4. Max Sharpe (Tangency Portfolio)](#p4)

### [4a. Calculate returns for Tangency Portfolio Strategy](#p4a)

# [5. Minimum Variance Portfolio](#p5)

### [5a. Calculate returns for Min Vol Portfolio Strategy](#p5a)

# [6. Target Return Portfolio](#p6)

### [6a. Calculate returns for Target Return Portfolio Strategy](#p6a)


# [7. Summary of strategy results](#p7)

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Yahoo finance
import yfinance as yf

# Pandas Datareader
import pandas_datareader as pdr

# Statsmodels
import statsmodels.api as sm

# PyPortfolioOpt
import pypfopt
from pypfopt import black_litterman, risk_models
from pypfopt import BlackLittermanModel
from pypfopt import EfficientFrontier
from pypfopt import objective_functions
from pypfopt import expected_returns
from pypfopt import plotting

# Datetime
import datetime as dt
from datetime import datetime
from datetime import timedelta

import warnings

warnings.filterwarnings("ignore")

In [2]:
# This is the start date to start downloading data
start = pd.to_datetime('2000-01-01', format='%Y-%m-%d')

# This is the start date to start trading.
trading_start_date = pd.to_datetime('2015-01-01', format='%Y-%m-%d')

# Assuming you run this in Singapore, you only have data until yesterday night (US time)
today_date = (datetime.today()- timedelta(days=1)).strftime('%Y-%m-%d') 

In [3]:
tickers = ["XLB","XLE","XLF","XLI","XLK","XLP","XLU","XLV","XLY","XHB","XRT","XSD","SPY","QQQ"]

# <a id = "p1">1.</a>  <font color = "green"> Functions </font>  [back to table of contents](#top)

In [4]:
def output_portfolio_statistics(portfolio_df: pd.DataFrame, 
                              SP500: pd.DataFrame, 
                              portfolio_name: str, 
                              mean_or_sum: str):
    
    # Ensure the index is a DatetimeIndex
    if not isinstance(portfolio_df.index, pd.DatetimeIndex):
        raise ValueError("The index of portfolio_df must be a DatetimeIndex")
    
    # Check if mean_or_sum is valid
    if mean_or_sum not in ["sum", "mean"]:
        raise ValueError("mean_or_sum must be 'sum' or 'mean'")

    portfolio_statistics=portfolio_df
    
    if portfolio_df is not SP500:
        # Get the common index between portfolio_statistics and SP500, ensures both dataframes have the same date range.
        common_index = SP500.index.intersection(portfolio_statistics.index)
        SP500 = SP500.loc[common_index]
        portfolio_statistics = portfolio_statistics.loc[common_index]
    
    #Code to calculate the portfolio's daily return, use sum if the portfolio is already weighted. If not weighted (equally weighted), use mean
    if mean_or_sum == "sum":
        portfolio_statistics['Returns']=portfolio_statistics.sum(axis=1,skipna=True)
    else:
        portfolio_statistics['Returns']=portfolio_statistics.mean(axis=1,skipna=True)
    
    portfolio_statistics.dropna(subset="Returns",axis=0,inplace=True)
    
    #Calculate number of years as the last date in the index minus the first date in the index
    Number_of_years=((portfolio_df.index[-1]-portfolio_df.index[0]).days)/365.25   #Edit this as needed
    
    # Cumulative returns, drawdown, and high-water mark
    portfolio_statistics['CumulativeStrategyReturns'] = (1+portfolio_statistics['Returns']).cumprod()
    portfolio_statistics['HighWaterMark'] = portfolio_statistics['CumulativeStrategyReturns'].cummax()
    portfolio_statistics['Drawdown'] = (portfolio_statistics['CumulativeStrategyReturns']/portfolio_statistics['HighWaterMark'])-1
    portfolio_statistics['MaxDrawdown'] = portfolio_statistics['Drawdown'].cummin()
    
    Last_date = portfolio_statistics[~portfolio_statistics['MaxDrawdown'].isna()].index[-1]
    
    # Convert index to a Series
    high_water_mark_date = pd.Series(portfolio_statistics.index, 
                                     index=portfolio_statistics.index)
    
    # Calculate HighWaterMarkDate vectorized
    portfolio_statistics['HighWaterMarkDate'] = high_water_mark_date.where(portfolio_statistics['Drawdown'] == 0).fillna(method='ffill')
            
    portfolio_statistics['HighWaterMarkDate'] = pd.to_datetime(portfolio_statistics['HighWaterMarkDate'])
    
    #Calculate duration of drawdown
    portfolio_statistics['DrawdownDuration'] = portfolio_statistics.index - portfolio_statistics['HighWaterMarkDate']

    #Calculate duration of maximum drawdown
    portfolio_statistics['MaximumDrawdownDuration'] = portfolio_statistics['DrawdownDuration'].cummax()
    
    #Calculate number of drawdowns during period of analysis
    portfolio_statistics['DrawdownNumber'] = 0
    
    # Counts the number of drawdowns.
    for row in range(1,portfolio_statistics.shape[0]):
        if (
            (portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc("Drawdown")] != 0) 
            and 
            (portfolio_statistics.iloc[row-1,portfolio_statistics.columns.get_loc("Drawdown")] == 0)
            ):
            portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc('DrawdownNumber')] = \
                (
                    portfolio_statistics.iloc[row-1,portfolio_statistics.columns.get_loc('DrawdownNumber')] + 1
                )
        else:
            portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc('DrawdownNumber')] = \
                (
                    portfolio_statistics.iloc[row-1,portfolio_statistics.columns.get_loc('DrawdownNumber')]
                )
    
    #Calculate ongoing max drawdown amount
    portfolio_statistics['OngoingMaxDrawdown'] = 0
    
    for row in range(1,portfolio_statistics.shape[0]):
        if (
            portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc('DrawdownNumber')] 
            != 
            portfolio_statistics.iloc[row-1,portfolio_statistics.columns.get_loc('DrawdownNumber')]
            ):
            portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc('OngoingMaxDrawdown')] = \
                (
                    portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc("Drawdown")]
                ) 
        else:
            portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc('OngoingMaxDrawdown')] = \
                (
                    min (portfolio_statistics.iloc[row-1,portfolio_statistics.columns.get_loc('OngoingMaxDrawdown')],
                         portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc("Drawdown")])
                )
    
    #Calculate average drawdown amount during period of analysis
    
    Drawdown_amount = 0
    
    for row in range(1,portfolio_statistics.shape[0]-1):
        if (
            portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc('DrawdownNumber')] 
            != 
            portfolio_statistics.iloc[row+1,portfolio_statistics.columns.get_loc('DrawdownNumber')]
            ):
            Drawdown_amount = \
                (
                    Drawdown_amount 
                    + 
                    portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc('OngoingMaxDrawdown')]
                )
        else:
            continue
        
    Average_drawdown_amount = \
        (
            Drawdown_amount
            /
            portfolio_statistics.iloc[-1,portfolio_statistics.columns.get_loc('DrawdownNumber')]
        )
        
    Average_drawdown_amount = round(Average_drawdown_amount*100,2)
    
    Cumulative_Returns=(portfolio_statistics['CumulativeStrategyReturns'][Last_date])

    Annualized_Returns=(portfolio_statistics['CumulativeStrategyReturns'][Last_date]**(1/Number_of_years))-1

    Annualized_Standard_Deviation = portfolio_statistics['Returns'].std() *np.sqrt(252)    #Edit this as needed

    Negative_Standard_Deviation = portfolio_statistics['Returns'][portfolio_statistics['Returns']<0].std()*np.sqrt(252)    #Edit this as needed

    Sharpe_Ratio = Annualized_Returns/Annualized_Standard_Deviation
    
    Sharpe_Upper_bound = round(Sharpe_Ratio + 1.96 * ((1+0.5*Sharpe_Ratio**2)/(portfolio_statistics.shape[0])),2)
    
    Sharpe_Lower_bound = round(Sharpe_Ratio - 1.96 * ((1+0.5*Sharpe_Ratio**2)/(portfolio_statistics.shape[0])),2)
    
    Sharpe_Ratio = round(Annualized_Returns/Annualized_Standard_Deviation,2)

    Sortino_Ratio = round(Annualized_Returns/Negative_Standard_Deviation,2)
    
    MaxDrawdown = round(portfolio_statistics['MaxDrawdown'][-1]*100,2)
    
    MaxDrawdown_Duration=(portfolio_statistics['MaximumDrawdownDuration'][Last_date]).days

    #Calculate VaR measures
    var99=np.percentile(portfolio_statistics['Returns'], 1)
    var99_month=round(var99 * np.sqrt(21),2)*100

    desired_stats_dict = {"Cumulative return (%)": round(Cumulative_Returns * 100,2),
                          "Annualized return (%)": round(Annualized_Returns * 100,2),
                          "Annualized Standard Deviation (%)": round(Annualized_Standard_Deviation * 100,2),
                          "Sharpe ratio": round(Sharpe_Ratio,2),
                          "Sortino ratio": round(Sortino_Ratio,2),
                          "Max drawdown (%)": round(MaxDrawdown,2),
                          "Max drawdown duration (days)": int(MaxDrawdown_Duration),
                          "1 month VaR (%)": round(var99_month,2)}

    return desired_stats_dict

In [5]:
# This is used to calculate portfolio statistics at the end
# Make sure SP500 and the portfolio df have the same number of rows. If not, information ratio and rolling beta will be wrong

def calc_portfolio_statistics(portfolio_df: pd.DataFrame, 
                              SP500: pd.DataFrame, 
                              portfolio_name: str, 
                              mean_or_sum: str):
    
    # Ensure the index is a DatetimeIndex
    if not isinstance(portfolio_df.index, pd.DatetimeIndex):
        raise ValueError("The index of portfolio_df must be a DatetimeIndex")
    
    # Check if mean_or_sum is valid
    if mean_or_sum not in ["sum", "mean"]:
        raise ValueError("mean_or_sum must be 'sum' or 'mean'")

    portfolio_statistics=portfolio_df
    
    if portfolio_df is not SP500:
        # Get the common index between portfolio_statistics and SP500, ensures both dataframes have the same date range.
        common_index = SP500.index.intersection(portfolio_statistics.index)
        SP500 = SP500.loc[common_index]
        portfolio_statistics = portfolio_statistics.loc[common_index]
    
    #Code to calculate the portfolio's daily return, use sum if the portfolio is already weighted. If not weighted (equally weighted), use mean
    if mean_or_sum == "sum":
        portfolio_statistics['Returns']=portfolio_statistics.sum(axis=1,skipna=True)
    else:
        portfolio_statistics['Returns']=portfolio_statistics.mean(axis=1,skipna=True)
    
    portfolio_statistics.dropna(subset="Returns",axis=0,inplace=True)
    
    #Calculate number of years as the last date in the index minus the first date in the index
    Number_of_years=((portfolio_df.index[-1]-portfolio_df.index[0]).days)/365.25   #Edit this as needed
    
    # Cumulative returns, drawdown, and high-water mark
    portfolio_statistics['CumulativeStrategyReturns'] = (1+portfolio_statistics['Returns']).cumprod()
    portfolio_statistics['HighWaterMark'] = portfolio_statistics['CumulativeStrategyReturns'].cummax()
    portfolio_statistics['Drawdown'] = (portfolio_statistics['CumulativeStrategyReturns']/portfolio_statistics['HighWaterMark'])-1
    portfolio_statistics['MaxDrawdown'] = portfolio_statistics['Drawdown'].cummin()
    
    Last_date = portfolio_statistics[~portfolio_statistics['MaxDrawdown'].isna()].index[-1]
    
    # Convert index to a Series
    high_water_mark_date = pd.Series(portfolio_statistics.index, 
                                     index=portfolio_statistics.index)
    
    # Calculate HighWaterMarkDate vectorized
    portfolio_statistics['HighWaterMarkDate'] = high_water_mark_date.where(portfolio_statistics['Drawdown'] == 0).fillna(method='ffill')
            
    portfolio_statistics['HighWaterMarkDate'] = pd.to_datetime(portfolio_statistics['HighWaterMarkDate'])
    
    #Calculate duration of drawdown
    portfolio_statistics['DrawdownDuration'] = portfolio_statistics.index - portfolio_statistics['HighWaterMarkDate']

    #Calculate duration of maximum drawdown
    portfolio_statistics['MaximumDrawdownDuration'] = portfolio_statistics['DrawdownDuration'].cummax()
    
    #Calculate number of drawdowns during period of analysis
    portfolio_statistics['DrawdownNumber'] = 0
    
    # Counts the number of drawdowns.
    for row in range(1,portfolio_statistics.shape[0]):
        if (
            (portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc("Drawdown")] != 0) 
            and 
            (portfolio_statistics.iloc[row-1,portfolio_statistics.columns.get_loc("Drawdown")] == 0)
            ):
            portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc('DrawdownNumber')] = \
                (
                    portfolio_statistics.iloc[row-1,portfolio_statistics.columns.get_loc('DrawdownNumber')] + 1
                )
        else:
            portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc('DrawdownNumber')] = \
                (
                    portfolio_statistics.iloc[row-1,portfolio_statistics.columns.get_loc('DrawdownNumber')]
                )
    
    #Calculate ongoing max drawdown amount
    portfolio_statistics['OngoingMaxDrawdown'] = 0
    
    for row in range(1,portfolio_statistics.shape[0]):
        if (
            portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc('DrawdownNumber')] 
            != 
            portfolio_statistics.iloc[row-1,portfolio_statistics.columns.get_loc('DrawdownNumber')]
            ):
            portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc('OngoingMaxDrawdown')] = \
                (
                    portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc("Drawdown")]
                ) 
        else:
            portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc('OngoingMaxDrawdown')] = \
                (
                    min (portfolio_statistics.iloc[row-1,portfolio_statistics.columns.get_loc('OngoingMaxDrawdown')],
                         portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc("Drawdown")])
                )
    
    #Calculate average drawdown amount during period of analysis
    
    Drawdown_amount = 0
    
    for row in range(1,portfolio_statistics.shape[0]-1):
        if (
            portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc('DrawdownNumber')] 
            != 
            portfolio_statistics.iloc[row+1,portfolio_statistics.columns.get_loc('DrawdownNumber')]
            ):
            Drawdown_amount = \
                (
                    Drawdown_amount 
                    + 
                    portfolio_statistics.iloc[row,portfolio_statistics.columns.get_loc('OngoingMaxDrawdown')]
                )
        else:
            continue
        
    Average_drawdown_amount = \
        (
            Drawdown_amount
            /
            portfolio_statistics.iloc[-1,portfolio_statistics.columns.get_loc('DrawdownNumber')]
        )
        
    Average_drawdown_amount = round(Average_drawdown_amount*100,2)
    
    Cumulative_Returns=(portfolio_statistics['CumulativeStrategyReturns'][Last_date])

    Annualized_Returns=(portfolio_statistics['CumulativeStrategyReturns'][Last_date]**(1/Number_of_years))-1

    Annualized_Standard_Deviation = portfolio_statistics['Returns'].std() *np.sqrt(252)    #Edit this as needed

    Negative_Standard_Deviation = portfolio_statistics['Returns'][portfolio_statistics['Returns']<0].std()*np.sqrt(252)    #Edit this as needed

    Sharpe_Ratio = Annualized_Returns/Annualized_Standard_Deviation
    
    Sharpe_Upper_bound = round(Sharpe_Ratio + 1.96 * ((1+0.5*Sharpe_Ratio**2)/(portfolio_statistics.shape[0])),2)
    
    Sharpe_Lower_bound = round(Sharpe_Ratio - 1.96 * ((1+0.5*Sharpe_Ratio**2)/(portfolio_statistics.shape[0])),2)
    
    Sharpe_Ratio = round(Annualized_Returns/Annualized_Standard_Deviation,2)

    Sortino_Ratio = round(Annualized_Returns/Negative_Standard_Deviation,2)
    
    Calmar_Ratio = round(Annualized_Returns/portfolio_statistics['MaxDrawdown'][-1]*-1,2)

    MaxDrawdown = round(portfolio_statistics['MaxDrawdown'][-1]*100,2)
    
    MaxDrawdown_Duration=(portfolio_statistics['MaximumDrawdownDuration'][Last_date]).days

    #Calculate VaR measures
    var99=np.percentile(portfolio_statistics['Returns'], 1)
    var99_month=round(var99 * np.sqrt(21),2)*100
    var99_year=round(var99 * np.sqrt(252),2)*100
    var99=round(var99,2)*100
    
    #Calculate Information Ratio
    Tracking_Error=(portfolio_statistics['Returns']-SP500['Returns']).std()*np.sqrt(252)
    SP500_Annualized_Returns=(SP500['CumulativeStrategyReturns'][Last_date]**(1/Number_of_years))-1
    Information_Ratio=round((Annualized_Returns-SP500_Annualized_Returns)/Tracking_Error,2)
    
    # For calculating beta
    stock_return = portfolio_statistics['Returns']
    SP500_return = SP500['Returns']

    # Compute rolling covariance and variance
    rolling_cov = stock_return.rolling(window=252).cov(SP500_return)
    rolling_var = SP500_return.rolling(window=252).var()   # Use SP500_return's variance

    # Calculate beta as covariance/variance
    beta = rolling_cov / rolling_var

    # Assign the result to beta_df
    portfolio_statistics["Rolling beta"] = beta
    
    Latest_beta=round(portfolio_statistics["Rolling beta"][-1],2)
    
    Treynor_Ratio=round(Annualized_Returns*100/Latest_beta,2)
    
    #Round values just before printing
    Cumulative_Returns=round(Cumulative_Returns*100,2)
    Annualized_Returns=round(Annualized_Returns*100,2)
    Annualized_Standard_Deviation=round(Annualized_Standard_Deviation*100,2)
    
    # Print the results
    print(f'During this period, {portfolio_name} Cumulative Return was {Cumulative_Returns}%')
    print(f'During this period, {portfolio_name} Annualized Return was {Annualized_Returns}%')
    print(f'During this period, {portfolio_name} Annualized Standard Deviation was {Annualized_Standard_Deviation}%')
    print(f'During this period, {portfolio_name} Sharpe Ratio was {Sharpe_Ratio}')
    print(f'During this period, {portfolio_name} Sharpe Ratio 95% confidence interval is between {Sharpe_Lower_bound} and {Sharpe_Upper_bound}')
    print(f'During this period, {portfolio_name} Sortino Ratio was {Sortino_Ratio}')
    print(f'During this period, {portfolio_name} Calmar Ratio was {Calmar_Ratio}')
    print(f'During this period, {portfolio_name} Treynor Ratio was {Treynor_Ratio}')
    print(f'During this period, {portfolio_name} Max Drawdown was {MaxDrawdown}%')
    print(f'During this period, {portfolio_name} Max Drawdown Duration was {MaxDrawdown_Duration} days')
    print(f'During this period, {portfolio_name} Average Drawdown was {Average_drawdown_amount}%')
    print(f'During this period, {portfolio_name} 1 day VaR at the 99% CI was {var99} %')
    print(f'During this period, {portfolio_name} 1 month VaR at the 99% CI was {var99_month} %')
    print(f'During this period, {portfolio_name} 1 year VaR at the 99% CI was {var99_year} %')
    print(f'During this period, {portfolio_name} Information Ratio against SP500 was {Information_Ratio}')
    print(f'The past 12m {portfolio_name} Rolling Beta against SP500 was {Latest_beta}')
    
    # Get the month and year
    portfolio_statistics['Month'] = portfolio_statistics.index.month
    portfolio_statistics['Year'] = portfolio_statistics.index.year
    
    # Define the aggregation for Returns
    agg_dict = {
        'Cumulative_Returns': ('Returns', lambda x: (1 + x).prod() - 1),
        'SD': ('Returns', 'std')
    }

    # Aggregate the statistics by Month and Year
    monthly_statistics = portfolio_statistics.groupby(["Month", "Year"]).agg(**agg_dict)

    # Calculate annualized metrics
    monthly_statistics["Annualized_Returns"] = (1 + monthly_statistics["Cumulative_Returns"]) ** 12 - 1
    monthly_statistics["Annualized_SD"] = monthly_statistics["SD"] * np.sqrt(12)
    monthly_statistics["Annualized_Sharpe"] = monthly_statistics["Annualized_Returns"] / monthly_statistics["Annualized_SD"].replace(0, np.nan)
    monthly_statistics.reset_index(inplace=True)
    monthly_statistics["Date"] = pd.to_datetime(monthly_statistics["Year"].astype(str) + "-" + monthly_statistics["Month"].astype(str) + "-01")
    monthly_statistics.sort_values(by="Date",ascending=True,inplace=True)
    monthly_statistics.set_index("Date",inplace=True)
    
    # Return the results
    return portfolio_statistics, monthly_statistics

In [6]:
#This is used to plot chart of cumulative returns

def plot_cumulative_return_chart(portfolio_df: pd.DataFrame, 
                                chart_title: str):
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=portfolio_df.index, 
        y=portfolio_df["Underlying_cumulative_returns"], 
        mode='lines', 
        name='Underlying Cumulative Returns'
    ))

    fig.add_trace(go.Scatter(
        x=portfolio_df.index, 
        y=portfolio_df["Strategy_cumulative_returns"], 
        mode='lines', 
        name='Strategy Cumulative Returns'
    ))

    fig.update_layout(
        title=chart_title,
        xaxis_title="Date",
        yaxis_title="Cumulative Returns",
        width=1200,  
        height=500
    )

    fig.show()

In [7]:
#This is used to plot chart of drawdowns

def plot_drawdown_chart(portfolio_df: pd.DataFrame, 
                        chart_title: str):
    # Create a figure with two line plots
    fig = make_subplots(specs=[[{"secondary_y": False}]])

    # Add the strategy's cumulative returns
    fig.add_trace(
        go.Scatter(
            x=portfolio_df.index,
            y=portfolio_df['Drawdown'], 
            mode='lines',
            name=chart_title,
            line=dict(color='green')
        )
    )

    # Update layout with title, labels, and legend
    fig.update_layout(
        title=chart_title,
        xaxis_title='Dates',
        yaxis_title='Returns',
        legend=dict(
            x=0.01, y=0.99,  # Position the legend
            bordercolor='Black', borderwidth=1
        ),
        template='plotly_white'
    )

    # Show the plot
    fig.show()

In [8]:
#This is used to plot chart of strategy return alongside S&P500

def plot_chart_with_SP500(portfolio_df: pd.DataFrame, 
                          chart_title: str):
    # Create a figure with two line plots
    fig = make_subplots(specs=[[{"secondary_y": False}]])

    # Add the strategy's cumulative returns
    fig.add_trace(
        go.Scatter(
            x=portfolio_df.index,
            y=portfolio_df, 
            mode='lines',
            name=chart_title,
            line=dict(color='green')
        )
    )

    # Add the S&P 500 cumulative returns
    fig.add_trace(
        go.Scatter(
            x=SP500.index,
            y=SP500['CumulativeStrategyReturns'], 
            mode='lines',
            name='S&P 500 Return',
            line=dict(color='red')
        )
    )

    # Update layout with title, labels, and legend
    fig.update_layout(
        title=chart_title,
        xaxis_title='Dates',
        yaxis_title='Returns',
        legend=dict(
            x=0.01, y=0.99,  # Position the legend
            bordercolor='Black', borderwidth=1
        ),
        template='plotly_white'
    )

    # Show the plot
    fig.show()

In [9]:
def plot_monthly_cumulative_returns(monthly_statistics_df: pd.DataFrame, 
                                    lookback_period: int,
                                    chart_title: str):
    
    """
    Plots the last `lookback_period` rows of the 'Cumulative_Returns' column with index as x-axis labels using Plotly.

    Parameters:
        monthly_statistics_df (pd.DataFrame): 
            The input DataFrame containing 'Cumulative_Returns' and index as date.
        lookback_period (int): 
            Number of months to look back for plotting.
    """
    
    # Select the last lookback_period rows
    last_months = monthly_statistics_df.tail(lookback_period).copy()
    
    # Convert cumulative returns to percentage
    last_months['Cumulative_Returns'] *= 100

    # Create bar plot for cumulative returns
    fig = px.bar(
        last_months,
        x=last_months.index,
        y='Cumulative_Returns',
        title=chart_title,
        labels={'x': 'Date', 'Cumulative_Returns': 'Monthly Returns (%)'}
    )

    # Customize layout
    fig.update_layout(
        xaxis_title='Date',
        yaxis_title='Monthly Returns (%)',
        xaxis_tickangle=-45,
        template='plotly_white'
    )

    # Show plot
    fig.show()

# <a id = "p2">2.</a>  <font color = "green"> Download data from yfinance </font>  [back to table of contents](#top)

In [10]:
Data_prices = yf.download(tickers, 
                     start=start, 
                     end=today_date,
                     auto_adjust=False)

[*********************100%***********************]  14 of 14 completed


In [11]:
Data_prices = Data_prices [["Adj Close"]]

In [12]:
Data_prices.columns = Data_prices.columns.get_level_values(level=1) # Keep the first level
Data_prices

Ticker,QQQ,SPY,XHB,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XRT,XSD
Date,,,,,,,,,,,,,,
2000-01-03,80.314301,92.414925,NaN,14.739728,13.739209,11.262168,18.318478,41.488525,12.474511,11.339868,20.805296,22.478333,NaN,NaN
2000-01-04,74.804604,88.800934,NaN,14.597837,13.480588,10.769833,17.812376,39.383686,12.123238,10.997803,20.333176,21.802118,NaN,NaN
2000-01-05,72.897408,88.959770,NaN,15.200895,13.836180,10.685211,17.732994,38.799019,12.337428,11.275328,20.150761,21.533968,NaN,NaN
2000-01-06,67.891037,87.530106,NaN,15.591128,14.369593,11.154475,17.971153,37.512737,12.568759,11.255962,20.215147,21.778807,NaN,NaN
2000-01-07,76.288025,92.613510,NaN,15.582260,14.523144,11.339098,18.655857,38.167568,13.365544,11.352785,20.461931,22.804783,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-28,472.410004,550.849976,93.769997,82.769997,82.970001,48.169998,129.440002,208.130005,80.570000,78.690002,138.600006,197.669998,68.669998,194.600006
2025-04-29,475.529999,554.320007,93.900002,83.459999,82.730003,48.669998,130.279999,209.100006,81.239998,79.300003,139.279999,199.070007,69.000000,191.550003
2025-04-30,475.470001,554.539978,94.699997,83.889999,80.500000,48.759998,131.220001,209.970001,81.830002,78.900002,140.470001,197.259995,68.699997,192.960007


In [13]:
# Data_prices.to_csv(path_or_buf="Data_prices.csv")

In [14]:
Data_returns = Data_prices/Data_prices.shift(1)-1

In [15]:
# Data_returns.to_csv(path_or_buf="Data_returns.csv")

# <a id = "p3">3.</a>  <font color = "green"> Preparing data for portfolio optimization </font>  [back to table of contents](#top)

https://github.com/robertmartin8/PyPortfolioOpt

https://github.com/robertmartin8/PyPortfolioOpt/blob/master/cookbook/2-Mean-Variance-Optimisation.ipynb

https://pyportfolioopt.readthedocs.io/en/latest/ExpectedReturns.html

https://pyportfolioopt.readthedocs.io/en/latest/RiskModels.html

https://pyportfolioopt.readthedocs.io/en/latest/GeneralEfficientFrontier.html

In [16]:
Trading_ETF = Data_prices[Data_prices.index>=trading_start_date]
Trading_ETF

Ticker,QQQ,SPY,XHB,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XRT,XSD
Date,,,,,,,,,,,,,,
2015-01-02,95.123276,172.075211,31.047009,39.578342,52.736538,16.526104,46.834198,36.375240,36.857216,33.962223,58.075417,63.773411,40.738506,37.880478
2015-01-05,93.727951,168.967621,30.716436,38.569569,50.554928,16.178604,45.740219,35.819973,36.598042,33.547009,57.779240,62.553677,40.277031,37.063999
2015-01-06,92.471199,167.376129,30.450129,38.227882,49.812260,15.931359,45.176647,35.388088,36.552299,33.568489,57.584637,61.930462,39.849739,36.157352
2015-01-07,93.663261,169.461807,31.285759,38.659046,49.918354,16.098413,45.516445,35.687756,37.177387,33.897797,58.938549,62.909824,40.892334,36.646278
2015-01-08,95.455910,172.468979,32.038750,39.578342,51.039001,16.338987,46.428097,36.472202,37.741489,34.134041,59.945557,63.853531,41.443554,37.780781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-28,472.410004,550.849976,93.769997,82.769997,82.970001,48.169998,129.440002,208.130005,80.570000,78.690002,138.600006,197.669998,68.669998,194.600006
2025-04-29,475.529999,554.320007,93.900002,83.459999,82.730003,48.669998,130.279999,209.100006,81.239998,79.300003,139.279999,199.070007,69.000000,191.550003
2025-04-30,475.470001,554.539978,94.699997,83.889999,80.500000,48.759998,131.220001,209.970001,81.830002,78.900002,140.470001,197.259995,68.699997,192.960007


In [17]:
Trading_ETF_Daily_Return_df=Trading_ETF/Trading_ETF.shift(1)-1
Trading_ETF_Daily_Return_df

Ticker,QQQ,SPY,XHB,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XRT,XSD
Date,,,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,-0.014669,-0.018059,-0.010647,-0.025488,-0.041368,-0.021027,-0.023359,-0.015265,-0.007032,-0.012226,-0.005100,-0.019126,-0.011328,-0.021554
2015-01-06,-0.013409,-0.009419,-0.008670,-0.008859,-0.014690,-0.015282,-0.012321,-0.012057,-0.001250,0.000640,-0.003368,-0.009963,-0.010609,-0.024462
2015-01-07,0.012891,0.012461,0.027443,0.011279,0.002130,0.010486,0.007522,0.008468,0.017101,0.009810,0.023512,0.015814,0.026163,0.013522
2015-01-08,0.019139,0.017745,0.024068,0.023780,0.022450,0.014944,0.020029,0.021981,0.015173,0.006969,0.017086,0.015001,0.013480,0.030958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-28,-0.000317,0.000381,0.001709,0.001815,0.007040,0.003124,0.003333,-0.001631,-0.003340,0.006137,0.003984,0.000354,0.002774,-0.000411
2025-04-29,0.006604,0.006299,0.001386,0.008336,-0.002893,0.010380,0.006489,0.004661,0.008316,0.007752,0.004906,0.007083,0.004806,-0.015673
2025-04-30,-0.000126,0.000397,0.008520,0.005152,-0.026955,0.001849,0.007215,0.004161,0.007262,-0.005044,0.008544,-0.009092,-0.004348,0.007361


In [18]:
Risk_free_rate = pdr.get_data_fred(['DGS1MO'],
                                 start=start,
                                 end=today_date)

Risk_free_rate = Risk_free_rate.rename(columns={'DGS1MO':'US_1M'})

Risk_free_rate

,US_1M
DATE,
2001-07-31,3.67
2001-08-01,3.65
2001-08-02,3.65
2001-08-03,3.63
2001-08-06,3.62
...,...
2025-04-25,4.34
2025-04-28,4.35
2025-04-29,4.35


### <a id = "p3a">3a.</a>  <font color = "green"> Need to get Third Thursday Dates to rebalance</font>  [back to table of contents](#top)

In [19]:
#Create a date dataframe to indentify the third thursday of every month
Date_df= pd.DataFrame(index=Trading_ETF.index)
Date_df.reset_index(inplace=True,names='Date')
Date_df['Date']= pd.to_datetime(Date_df['Date'])
Date_df['Month'] = Date_df['Date'].dt.month
Date_df['Year'] = Date_df['Date'].dt.year
Date_df['Weekday'] = Date_df['Date'].dt.weekday

In [20]:
#Check which are the third Thursdays of the month

Date_df['Third_Thursday']=0                                    # We are adding one extra column here
count=0

for row in range (len(Date_df)):
        if Date_df.iloc[row,-4] == Date_df.iloc[row-1,-4]:     # If month is equal to previous row's month
            if Date_df.iloc[row,-2] == 3:                      # If weekday is Thursday. Check which column weekday is 
                count += 1
                if count==3:                                   # If third Thursday in the month
                        Date_df.iloc[row,-1]=1                 # Assign -1 to the last column
                else:
                    continue
            else:
                continue
        else:                                                  # If it is a new month
            if Date_df.iloc[row,-2] == 3:                      # If weekday is Thursday. Check which column weekday is 
                count = 1
            else: 
                count=0

In [21]:
#Get the dates which are third Thursdays of the month as a string

Third_Thursday_Dates=[]

for row in range (len(Date_df)):
    if Date_df.iloc[row,-1] == 1:
        Third_Thursday_Dates.append(Date_df.iloc[row,-5])
        
Third_Thursday_Dates

[Timestamp('2015-01-22 00:00:00'),
 Timestamp('2015-02-19 00:00:00'),
 Timestamp('2015-03-19 00:00:00'),
 Timestamp('2015-04-16 00:00:00'),
 Timestamp('2015-05-21 00:00:00'),
 Timestamp('2015-06-18 00:00:00'),
 Timestamp('2015-07-16 00:00:00'),
 Timestamp('2015-08-20 00:00:00'),
 Timestamp('2015-09-17 00:00:00'),
 Timestamp('2015-10-15 00:00:00'),
 Timestamp('2015-11-19 00:00:00'),
 Timestamp('2015-12-17 00:00:00'),
 Timestamp('2016-01-21 00:00:00'),
 Timestamp('2016-02-18 00:00:00'),
 Timestamp('2016-03-17 00:00:00'),
 Timestamp('2016-04-21 00:00:00'),
 Timestamp('2016-05-19 00:00:00'),
 Timestamp('2016-06-16 00:00:00'),
 Timestamp('2016-07-21 00:00:00'),
 Timestamp('2016-08-18 00:00:00'),
 Timestamp('2016-09-15 00:00:00'),
 Timestamp('2016-10-20 00:00:00'),
 Timestamp('2016-11-17 00:00:00'),
 Timestamp('2016-12-15 00:00:00'),
 Timestamp('2017-01-19 00:00:00'),
 Timestamp('2017-02-16 00:00:00'),
 Timestamp('2017-03-16 00:00:00'),
 Timestamp('2017-04-20 00:00:00'),
 Timestamp('2017-05-

### <a id = "p3b">3b.</a>  <font color = "green"> Need to get S&P to get market returns</font>  [back to table of contents](#top)

In [22]:
#Get data from S&P so that we can compare

SP500 = yf.download('^GSPC',
                    start=Third_Thursday_Dates[0] - timedelta(days=1),
                    end=today_date,
                    interval='1D',
                    auto_adjust=False)

[*********************100%***********************]  1 of 1 completed


In [23]:
SP500_for_MVO = SP500[["Adj Close"]]

In [24]:
SP500.reset_index(inplace=True)
SP500['DailyReturns']=(SP500['Adj Close']/SP500['Adj Close'].shift(1))-1
SP500= SP500[SP500['Date'] >= trading_start_date]
SP500.set_index('Date',inplace=True)

In [25]:
# We only need DailyReturns
SP500=SP500['DailyReturns']
SP500=SP500.to_frame()

In [26]:
SP500, SP500_monthly_statistics = calc_portfolio_statistics(portfolio_df=SP500, 
                                                            SP500=SP500, 
                                                            portfolio_name="SP500", 
                                                            mean_or_sum="mean")

During this period, SP500 Cumulative Return was 279.84%
During this period, SP500 Annualized Return was 10.53%
During this period, SP500 Annualized Standard Deviation was 18.25%
During this period, SP500 Sharpe Ratio was 0.58
During this period, SP500 Sharpe Ratio 95% confidence interval is between 0.58 and 0.58
During this period, SP500 Sortino Ratio was 0.7
During this period, SP500 Calmar Ratio was 0.31
During this period, SP500 Treynor Ratio was 10.53
During this period, SP500 Max Drawdown was -33.92%
During this period, SP500 Max Drawdown Duration was 745 days
During this period, SP500 Average Drawdown was -1.71%
During this period, SP500 1 day VaR at the 99% CI was -3.0 %
During this period, SP500 1 month VaR at the 99% CI was -15.0 %
During this period, SP500 1 year VaR at the 99% CI was -53.0 %
During this period, SP500 Information Ratio against SP500 was nan
The past 12m SP500 Rolling Beta against SP500 was 1.0


In [27]:
SP500

,DailyReturns,Returns,CumulativeStrategyReturns,HighWaterMark,Drawdown,MaxDrawdown,HighWaterMarkDate,DrawdownDuration,MaximumDrawdownDuration,DrawdownNumber,OngoingMaxDrawdown,Rolling beta,Month,Year
Date,,,,,,,,,,,,,,
2015-01-22,0.015270,0.015270,1.015270,1.015270,0.000000,0.000000,2015-01-22,0 days,0 days,0,0.000000,NaN,1,2015
2015-01-23,-0.005492,-0.005492,1.009694,1.015270,-0.005492,-0.005492,2015-01-22,1 days,1 days,1,-0.005492,NaN,1,2015
2015-01-26,0.002568,0.002568,1.012288,1.015270,-0.002937,-0.005492,2015-01-22,4 days,4 days,1,-0.005492,NaN,1,2015
2015-01-27,-0.013388,-0.013388,0.998735,1.015270,-0.016286,-0.016286,2015-01-22,5 days,5 days,1,-0.016286,NaN,1,2015
2015-01-28,-0.013496,-0.013496,0.985257,1.015270,-0.029562,-0.029562,2015-01-22,6 days,6 days,1,-0.029562,NaN,1,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-28,0.000641,0.000641,2.720681,3.023517,-0.100160,-0.339250,2025-02-19,68 days,745 days,158,-0.189022,1.0,4,2025
2025-04-29,0.005802,0.005802,2.736467,3.023517,-0.094939,-0.339250,2025-02-19,69 days,745 days,158,-0.189022,1.0,4,2025
2025-04-30,0.001480,0.001480,2.740517,3.023517,-0.093600,-0.339250,2025-02-19,70 days,745 days,158,-0.189022,1.0,4,2025


In [28]:
SP500_monthly_statistics

,Month,Year,Cumulative_Returns,SD,Annualized_Returns,Annualized_SD,Annualized_Sharpe
Date,,,,,,,
2015-01-01,1,2015,-0.018272,0.011865,-0.198513,0.041103,-4.829642
2015-02-01,2,2015,0.054893,0.006060,0.898884,0.020991,42.821478
2015-03-01,3,2015,-0.017396,0.009018,-0.189894,0.031238,-6.078872
2015-04-01,4,2015,0.008521,0.005342,0.107180,0.018504,5.792204
2015-05-01,5,2015,0.010491,0.006810,0.133421,0.023590,5.655916
...,...,...,...,...,...,...,...
2025-01-01,1,2025,0.027016,0.008847,0.376981,0.030646,12.300981
2025-02-01,2,2025,-0.014242,0.008336,-0.158133,0.028878,-5.475966
2025-03-01,3,2025,-0.057545,0.012990,-0.508946,0.044997,-11.310589


In [29]:
plot_monthly_cumulative_returns(monthly_statistics_df = SP500_monthly_statistics,
                                lookback_period = 24,
                                chart_title= " Last 24 Months of returns of SP500")

In [30]:
plot_drawdown_chart(portfolio_df = SP500, 
                        chart_title = "SP500 Drawdown")

### <a id = "p3c">3c.</a>  <font color = "green"> Creating inputs for MVO</font>  [back to table of contents](#top)

In [31]:
# This is for recording the risk-free rate 5years before they are traded
# Dictionary to store the split DataFrames

historical_risk_free_rate = {}
n=1

# Iterate over each date in Third_Thursday_Dates
for date_counter, date_name in enumerate(Third_Thursday_Dates):
    # Slice DataFrame by date
    Risk_free_rate_slice = Risk_free_rate.loc[date_name]

    # Store in dictionary with a dynamic key
    historical_risk_free_rate[n] = Risk_free_rate_slice
    
    n=n+1
    
historical_risk_free_rate

{1: US_1M    0.02
 Name: 2015-01-22 00:00:00, dtype: float64,
 2: US_1M    0.02
 Name: 2015-02-19 00:00:00, dtype: float64,
 3: US_1M    0.02
 Name: 2015-03-19 00:00:00, dtype: float64,
 4: US_1M    0.03
 Name: 2015-04-16 00:00:00, dtype: float64,
 5: US_1M    0.02
 Name: 2015-05-21 00:00:00, dtype: float64,
 6: US_1M    0.0
 Name: 2015-06-18 00:00:00, dtype: float64,
 7: US_1M    0.03
 Name: 2015-07-16 00:00:00, dtype: float64,
 8: US_1M    0.01
 Name: 2015-08-20 00:00:00, dtype: float64,
 9: US_1M    0.0
 Name: 2015-09-17 00:00:00, dtype: float64,
 10: US_1M    0.0
 Name: 2015-10-15 00:00:00, dtype: float64,
 11: US_1M    0.06
 Name: 2015-11-19 00:00:00, dtype: float64,
 12: US_1M    0.18
 Name: 2015-12-17 00:00:00, dtype: float64,
 13: US_1M    0.27
 Name: 2016-01-21 00:00:00, dtype: float64,
 14: US_1M    0.28
 Name: 2016-02-18 00:00:00, dtype: float64,
 15: US_1M    0.29
 Name: 2016-03-17 00:00:00, dtype: float64,
 16: US_1M    0.19
 Name: 2016-04-21 00:00:00, dtype: float64,
 17:

In [32]:
# This is for recording the prices of SP500 5years before they are traded
# Dictionary to store the split DataFrames

SP500_historical_prices = {}
n=1

# Iterate over each date in Third_Thursday_Dates
for date_counter, date_name in enumerate(Third_Thursday_Dates):
    start_date = date_name - timedelta(days=1825)   #default is 1825
    end_date = date_name
    # Slice DataFrame by date range
    SP500_price_slice = SP500_for_MVO.loc[start_date:end_date]

    # Store in dictionary with a dynamic key
    SP500_historical_prices[n] = SP500_price_slice
    
    n=n+1

In [33]:
# This is for recording the returns of the ETFs 5 years before they are traded
# Dictionary to store the split DataFrames

ETF_historical_returns = {}
n=1

# Iterate over each date in Third_Thursday_Dates
for date_counter, date_name in enumerate(Third_Thursday_Dates):
    Returns_start_date = date_name - timedelta(days=1825)
    Returns_end_date = date_name
    # Slice DataFrame by date range
    ETF_return_slice = Data_returns.loc[Returns_start_date:Returns_end_date]
        
    # Store in dictionary with a dynamic key
    ETF_historical_returns[n] = ETF_return_slice

    n=n+1

In [34]:
# This is for recording the prices of the ETFs 5 years before they are traded
# Dictionary to store the split DataFrames

ETF_historical_prices = {}
n=1

# Iterate over each date in Third_Thursday_Dates
for date_counter, date_name in enumerate(Third_Thursday_Dates):
    Prices_start_date = date_name - timedelta(days=1825)
    Prices_end_date = date_name
    # Slice DataFrame by date range
    ETF_price_slice = Data_prices.loc[Prices_start_date:Prices_end_date]
        
    # Store in dictionary with a dynamic key
    ETF_historical_prices[n] = ETF_price_slice
    
    n=n+1

In [35]:
ETF_expected_returns = {}

for key,value in ETF_historical_prices.items():
    
    # Can choose whether to put in market prices or not to calculate capm return
    #mu = expected_returns.capm_return(value,market_prices=SP500_historical_prices[key],risk_free_rate=0.0)
    mu = expected_returns.capm_return(value,risk_free_rate=float(historical_risk_free_rate[key]/100))
    
    ETF_expected_returns[key]=mu

In [36]:
ETF_expected_returns

{1: Ticker
 QQQ    0.162491
 SPY    0.157204
 XHB    0.227105
 XLB    0.196202
 XLE    0.192624
 XLF    0.200674
 XLI    0.181891
 XLK    0.153091
 XLP    0.095416
 XLU    0.093099
 XLV    0.126364
 XLY    0.166627
 XRT    0.183876
 XSD    0.226682
 Name: mkt, dtype: float64,
 2: Ticker
 QQQ    0.163880
 SPY    0.158646
 XHB    0.229252
 XLB    0.197141
 XLE    0.193800
 XLF    0.202463
 XLI    0.183346
 XLK    0.154398
 XLP    0.096582
 XLU    0.093912
 XLV    0.127842
 XLY    0.168503
 XRT    0.185649
 XSD    0.227506
 Name: mkt, dtype: float64,
 3: Ticker
 QQQ    0.151104
 SPY    0.146557
 XHB    0.211007
 XLB    0.181736
 XLE    0.178613
 XLF    0.186629
 XLI    0.169473
 XLK    0.142582
 XLP    0.089795
 XLU    0.087468
 XLV    0.118491
 XLY    0.155557
 XRT    0.170516
 XSD    0.208885
 Name: mkt, dtype: float64,
 4: Ticker
 QQQ    0.146820
 SPY    0.142133
 XHB    0.204228
 XLB    0.175904
 XLE    0.172655
 XLF    0.180487
 XLI    0.164373
 XLK    0.138476
 XLP    0.087291
 XLU 

# <a id = "p4">4.</a>  <font color = "green"> Max Sharpe (Tangency Portfolio)</font>  [back to table of contents](#top)

In [37]:
ETF_mvo_weights_dict = {}

for key,value in ETF_historical_prices.items():
    
    try:

        S = risk_models.CovarianceShrinkage(value).ledoit_wolf()
        
        # You can provide expected returns if you want to
        ef = EfficientFrontier(ETF_expected_returns[key], S, weight_bounds=(0, 1))   # can cap the weights here
        
        # Choose your target
        #ef.min_volatility()
        ef.max_sharpe(risk_free_rate=float(historical_risk_free_rate[key]/100))   # Note: For max Sharpe, you probably would not want to provide expected returns 
        #ef.efficient_return(target_return=0.07, market_neutral=True)
        #ef.efficient_risk(target_volatility=0.35)
        
        weights = ef.clean_weights()
        
        ETF_mvo_weights_dict[key] = weights
        
    except:
        
        continue

In [38]:
ETF_mvo_weights_dict

{1: OrderedDict([('QQQ', 0.07183),
              ('SPY', 0.07111),
              ('XHB', 0.07166),
              ('XLB', 0.07158),
              ('XLE', 0.07163),
              ('XLF', 0.0719),
              ('XLI', 0.07178),
              ('XLK', 0.07095),
              ('XLP', 0.07055),
              ('XLU', 0.07114),
              ('XLV', 0.07133),
              ('XLY', 0.07142),
              ('XRT', 0.07153),
              ('XSD', 0.07159)]),
 2: OrderedDict([('QQQ', 0.07183),
              ('SPY', 0.07112),
              ('XHB', 0.07165),
              ('XLB', 0.07158),
              ('XLE', 0.0716),
              ('XLF', 0.07191),
              ('XLI', 0.07178),
              ('XLK', 0.07095),
              ('XLP', 0.07055),
              ('XLU', 0.07117),
              ('XLV', 0.07133),
              ('XLY', 0.07141),
              ('XRT', 0.07153),
              ('XSD', 0.07159)]),
 3: OrderedDict([('QQQ', 0.07181),
              ('SPY', 0.07109),
              ('XHB', 0.07165

In [39]:
MVO_optimized_weights=pd.DataFrame(index=Third_Thursday_Dates, 
                                   columns=Data_prices.columns)

for row in range (len(MVO_optimized_weights)):
    for column in range (MVO_optimized_weights.shape[1]):
        try:
            column_name=MVO_optimized_weights.columns[column]
            row_number=row+1
            MVO_optimized_weights.iloc[row,column] =  ETF_mvo_weights_dict[row_number] [column_name]
        except Exception as e:
            print(f"Error at row {row}, column {column}: {e}")
            
MVO_optimized_weights

Ticker,QQQ,SPY,XHB,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XRT,XSD
2015-01-22,0.07183,0.07111,0.07166,0.07158,0.07163,0.0719,0.07178,0.07095,0.07055,0.07114,0.07133,0.07142,0.07153,0.07159
2015-02-19,0.07183,0.07112,0.07165,0.07158,0.0716,0.07191,0.07178,0.07095,0.07055,0.07117,0.07133,0.07141,0.07153,0.07159
2015-03-19,0.07181,0.07109,0.07165,0.07157,0.07159,0.07189,0.07178,0.07099,0.07057,0.0712,0.07133,0.07143,0.07151,0.07158
2015-04-16,0.07181,0.07109,0.07165,0.07157,0.07159,0.0719,0.07178,0.07099,0.07057,0.0712,0.07132,0.07145,0.0715,0.07158
2015-05-21,0.07179,0.07107,0.07165,0.07157,0.07159,0.07188,0.07176,0.07102,0.07055,0.07121,0.07137,0.07144,0.0715,0.07158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-19,0.07106,0.06997,0.07218,0.07154,0.07163,0.07252,0.07112,0.07308,0.06966,0.07199,0.07049,0.07138,0.07159,0.07177
2025-01-23,0.07105,0.06996,0.07216,0.07154,0.07163,0.07251,0.07116,0.07307,0.06966,0.07199,0.07051,0.0714,0.07159,0.07177
2025-02-20,0.07107,0.06989,0.0722,0.07154,0.07163,0.07255,0.07109,0.07301,0.06973,0.07198,0.07052,0.07146,0.07158,0.07177
2025-03-20,0.0711,0.07044,0.07201,0.07165,0.07158,0.07198,0.07118,0.07237,0.0701,0.07166,0.07096,0.07173,0.07155,0.07167


In [40]:
MVO_optimized_weights.to_csv(path_or_buf="MVO_optimized_weights.csv")

In [41]:
#The weights will only be realized in the returns tomorrow. Today's return depends on yesterday's weight

MVO_daily_weights_df = pd.DataFrame(index = Trading_ETF.index, 
                                     columns = MVO_optimized_weights.columns)

# Iterate through rows of MVO_daily_weights_df 
for row in range(len(MVO_daily_weights_df )):
    # Iterate through rows of MVO_optimized_weights
    for row_1 in range(len(MVO_optimized_weights) - 1):  # Use len(MVO_optimized_weights) - 1 to avoid index out of range
        # Compare row names and assign values
        if (
            (MVO_daily_weights_df .index[row] >= MVO_optimized_weights.index[row_1]) 
            and 
            (MVO_daily_weights_df .index[row] < MVO_optimized_weights.index[row_1 + 1])
            ):
            MVO_daily_weights_df .iloc[row] = MVO_optimized_weights.iloc[row_1]
            break  # Exit inner loop after assigning once (assuming you want only one assignment per row)

    # Handle the last row of MVO_optimized_weights separately
    last_row_index = len(MVO_optimized_weights) - 1
    if MVO_daily_weights_df .index[row] >= MVO_optimized_weights.index[last_row_index]:
        MVO_daily_weights_df .iloc[row] = MVO_optimized_weights.iloc[last_row_index]
        
MVO_daily_weights_df

Ticker,QQQ,SPY,XHB,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XRT,XSD
Date,,,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-28,0.07129,0.07108,0.07179,0.07173,0.07151,0.07182,0.07126,0.07191,0.07038,0.07146,0.07086,0.07165,0.07155,0.0717
2025-04-29,0.07129,0.07108,0.07179,0.07173,0.07151,0.07182,0.07126,0.07191,0.07038,0.07146,0.07086,0.07165,0.07155,0.0717
2025-04-30,0.07129,0.07108,0.07179,0.07173,0.07151,0.07182,0.07126,0.07191,0.07038,0.07146,0.07086,0.07165,0.07155,0.0717


### <a id = "p4a">4a.</a>  <font color = "green"> Calculate returns for Tangency Portfolio Strategy</font>  [back to table of contents](#top)

In [42]:
# Extract the first valid index from MVO_daily_weights_df
first_valid_index = MVO_daily_weights_df.first_valid_index()

# Filter Trading_ETF_Daily_Return_df based on this index
Trading_ETF_Daily_Return_df = Trading_ETF_Daily_Return_df[Trading_ETF_Daily_Return_df.index >= first_valid_index]

# Filter MVO_daily_weights_df based on this index
MVO_daily_weights_df = MVO_daily_weights_df[MVO_daily_weights_df.index >= first_valid_index]

In [43]:
# In this version, Daily_ETF_Portfolio_returns cannot be negative as it is esentially (weights * (1+returns)) and weights cannot be negative

# Initialize a DataFrame for portfolio returns
Daily_MVO_ETF_Portfolio_returns = pd.DataFrame(
    index=MVO_daily_weights_df.index, 
    columns=MVO_daily_weights_df.columns
)

# Loop through columns and rows to compute portfolio returns
for column in range(MVO_daily_weights_df.shape[1]):                           
    for row in range(1, len(MVO_daily_weights_df)):
        # Retrieve the row name (date) and column name
        row_name = MVO_daily_weights_df.index[row]
        column_name = MVO_daily_weights_df.columns[column]

        # Ensure valid data retrieval and handle third Thursday logic
        if Daily_MVO_ETF_Portfolio_returns.index[row - 1] in Third_Thursday_Dates:
            # Apply new weights multiplied by daily returns
            Daily_MVO_ETF_Portfolio_returns.iloc[row, column] = (
                MVO_daily_weights_df.iloc[row - 1, column] * 
                (1 + Trading_ETF_Daily_Return_df.loc[row_name, column_name])
            )
        else:
            # Multiply cumulative returns otherwise
            Daily_MVO_ETF_Portfolio_returns.iloc[row, column] = (
                Daily_MVO_ETF_Portfolio_returns.iloc[row - 1, column] * 
                (1 + Trading_ETF_Daily_Return_df.loc[row_name, column_name])
            )

# Calculate the total portfolio return by summing across ETF columns
Daily_MVO_ETF_Portfolio_returns['Total'] = Daily_MVO_ETF_Portfolio_returns.sum(axis=1)

Daily_MVO_ETF_Portfolio_returns

Ticker,QQQ,SPY,XHB,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XRT,XSD,Total
Date,,,,,,,,,,,,,,,
2015-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2015-01-23,0.071989,0.07072,0.071618,0.07047,0.07112,0.071122,0.071183,0.070933,0.069763,0.071343,0.07092,0.071319,0.071999,0.071896,0.996394
2015-01-26,0.071906,0.070886,0.07274,0.070777,0.07214,0.071481,0.071271,0.070642,0.069763,0.0713,0.07129,0.071692,0.072604,0.072543,1.001034
2015-01-27,0.070042,0.069951,0.072846,0.070295,0.072094,0.070703,0.070331,0.068606,0.068962,0.071401,0.07078,0.071118,0.07237,0.072057,0.991555
2015-01-28,0.069683,0.069054,0.072698,0.069024,0.069255,0.069417,0.06972,0.068486,0.068161,0.070792,0.06976,0.070193,0.071205,0.071842,0.979289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-28,0.075835,0.07438,0.074097,0.073524,0.0728,0.07416,0.073614,0.077604,0.069262,0.072305,0.072487,0.076392,0.073885,0.080745,1.041091
2025-04-29,0.076335,0.074849,0.0742,0.074137,0.072589,0.07493,0.074092,0.077965,0.069838,0.072866,0.072842,0.076933,0.07424,0.07948,1.045297
2025-04-30,0.076326,0.074878,0.074832,0.074519,0.070633,0.075068,0.074627,0.07829,0.070346,0.072498,0.073464,0.076233,0.073917,0.080065,1.045697


In [44]:
CumulativeLongMVOStrategyReturnWithoutSL= pd.DataFrame(index=Daily_MVO_ETF_Portfolio_returns.index,
                                                    columns=['Average'])

for row in range(1,len(CumulativeLongMVOStrategyReturnWithoutSL)):
    if CumulativeLongMVOStrategyReturnWithoutSL.index[row-1] in Third_Thursday_Dates:                      # If yesterday is Third Thursday
        CumulativeLongMVOStrategyReturnWithoutSL.iloc[row,:]=Daily_MVO_ETF_Portfolio_returns.iloc[row,-1]-1    # The return series starts anew
    else:
        CumulativeLongMVOStrategyReturnWithoutSL.iloc[row,:] = \
            (
                (
                    Daily_MVO_ETF_Portfolio_returns.iloc[row,-1]      #If yesterday is not Third Thursday, need to divide to obtain the daily return
                    /
                    Daily_MVO_ETF_Portfolio_returns.iloc[row-1,-1]
                 )-1  
            )
            
CumulativeLongMVOStrategyReturnWithoutSL

,Average
Date,
2015-01-22,NaN
2015-01-23,-0.003606
2015-01-26,0.004657
2015-01-27,-0.009469
2015-01-28,-0.012371
...,...
2025-04-28,0.001748
2025-04-29,0.00404
2025-04-30,0.000383


In [45]:
MVO_performance_df, MVO_monthly_statistics=calc_portfolio_statistics(portfolio_df=CumulativeLongMVOStrategyReturnWithoutSL, 
                                                                    SP500=SP500, 
                                                                    portfolio_name="MVO, no SL",
                                                                    mean_or_sum="mean")

During this period, MVO, no SL Cumulative Return was 318.67%
During this period, MVO, no SL Annualized Return was 11.94%
During this period, MVO, no SL Annualized Standard Deviation was 18.66%
During this period, MVO, no SL Sharpe Ratio was 0.64
During this period, MVO, no SL Sharpe Ratio 95% confidence interval is between 0.64 and 0.64
During this period, MVO, no SL Sortino Ratio was 0.79
During this period, MVO, no SL Calmar Ratio was 0.33
During this period, MVO, no SL Treynor Ratio was 12.7
During this period, MVO, no SL Max Drawdown was -36.55%
During this period, MVO, no SL Max Drawdown Duration was 560 days
During this period, MVO, no SL Average Drawdown was -1.92%
During this period, MVO, no SL 1 day VaR at the 99% CI was -3.0 %
During this period, MVO, no SL 1 month VaR at the 99% CI was -15.0 %
During this period, MVO, no SL 1 year VaR at the 99% CI was -51.0 %
During this period, MVO, no SL Information Ratio against SP500 was 0.37
The past 12m MVO, no SL Rolling Beta against

In [46]:
MVO_performance_df

,Average,Returns,CumulativeStrategyReturns,HighWaterMark,Drawdown,MaxDrawdown,HighWaterMarkDate,DrawdownDuration,MaximumDrawdownDuration,DrawdownNumber,OngoingMaxDrawdown,Rolling beta,Month,Year
Date,,,,,,,,,,,,,,
2015-01-23,-0.003606,-0.003606,0.996394,0.996394,0.0,0.0,2015-01-23,0 days,0 days,0,0.000000,NaN,1,2015
2015-01-26,0.004657,0.004657,1.001034,1.001034,0.0,0.0,2015-01-26,0 days,0 days,0,0.000000,NaN,1,2015
2015-01-27,-0.009469,-0.009469,0.991555,1.001034,-0.009469,-0.009469,2015-01-26,1 days,1 days,1,-0.009469,NaN,1,2015
2015-01-28,-0.012371,-0.012371,0.979289,1.001034,-0.021723,-0.021723,2015-01-26,2 days,2 days,1,-0.021723,NaN,1,2015
2015-01-29,0.01052,0.01052,0.98959,1.001034,-0.011432,-0.021723,2015-01-26,3 days,3 days,1,-0.021723,NaN,1,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-28,0.001748,0.001748,3.108003,3.460069,-0.101751,-0.365462,2024-12-02,147 days,560 days,161,-0.188710,0.937499,4,2025
2025-04-29,0.00404,0.00404,3.120561,3.460069,-0.098122,-0.365462,2024-12-02,148 days,560 days,161,-0.188710,0.938012,4,2025
2025-04-30,0.000383,0.000383,3.121754,3.460069,-0.097777,-0.365462,2024-12-02,149 days,560 days,161,-0.188710,0.939072,4,2025


In [47]:
MVO_monthly_statistics

,Month,Year,Cumulative_Returns,SD,Annualized_Returns,Annualized_SD,Annualized_Sharpe
Date,,,,,,,
2015-01-01,1,2015,-0.023636,0.009732,-0.249514,0.033714,-7.400982
2015-02-01,2,2015,0.055822,0.005979,0.919054,0.020713,44.370044
2015-03-01,3,2015,-0.010588,0.008810,-0.119912,0.030520,-3.928947
2015-04-01,4,2015,-0.000732,0.005435,-0.008745,0.018827,-0.464469
2015-05-01,5,2015,0.016639,0.007033,0.218989,0.024363,8.988651
...,...,...,...,...,...,...,...
2025-01-01,1,2025,0.030441,0.007813,0.433095,0.027065,16.001970
2025-02-01,2,2025,-0.017591,0.008192,-0.191819,0.028376,-6.759839
2025-03-01,3,2025,-0.043586,0.011996,-0.414197,0.041556,-9.967213


In [48]:
plot_chart_with_SP500(portfolio_df=MVO_performance_df['CumulativeStrategyReturns'],
                      chart_title=f"MVO Strategy Return")

In [49]:
plot_monthly_cumulative_returns(monthly_statistics_df = MVO_monthly_statistics,
                                lookback_period = 24,
                                chart_title= " Last 24 Months of returns of MVO Portfolio")

In [50]:
plot_drawdown_chart(portfolio_df = MVO_performance_df, 
                    chart_title = f"MVO Portfolio Drawdown")

# <a id = "p5">5.</a>  <font color = "green"> Minimum Variance Portfolio</font>  [back to table of contents](#top)

In [51]:
ETF_min_vol_weights_dict = {}

for key,value in ETF_historical_prices.items():
    
    try:

        S = risk_models.CovarianceShrinkage(value).ledoit_wolf()
        
        # You can provide expected returns if you want to
        ef = EfficientFrontier(ETF_expected_returns[key], S, weight_bounds=(0, 1))   # can cap the weights here
        
        # Choose your target
        ef.min_volatility()
        #ef.max_sharpe(risk_free_rate=historical_risk_free_rate[key])   # Note: For max Sharpe, you probably would not want to provide expected returns 
        #ef.efficient_return(target_return=0.07, market_neutral=True)
        #ef.efficient_risk(target_volatility=0.35)
        
        weights = ef.clean_weights()
        
        ETF_min_vol_weights_dict[key] = weights
        
    except:
        
        continue

In [52]:
ETF_min_vol_weights_dict

{1: OrderedDict([('QQQ', 0.0),
              ('SPY', 0.0),
              ('XHB', 0.0),
              ('XLB', 0.0),
              ('XLE', 0.0),
              ('XLF', 0.0),
              ('XLI', 0.0),
              ('XLK', 0.0),
              ('XLP', 0.75049),
              ('XLU', 0.24951),
              ('XLV', 0.0),
              ('XLY', 0.0),
              ('XRT', 0.0),
              ('XSD', 0.0)]),
 2: OrderedDict([('QQQ', 0.0),
              ('SPY', 0.0),
              ('XHB', 0.0),
              ('XLB', 0.0),
              ('XLE', 0.0),
              ('XLF', 0.0),
              ('XLI', 0.0),
              ('XLK', 0.0),
              ('XLP', 0.76227),
              ('XLU', 0.23773),
              ('XLV', 0.0),
              ('XLY', 0.0),
              ('XRT', 0.0),
              ('XSD', 0.0)]),
 3: OrderedDict([('QQQ', 0.0),
              ('SPY', 0.0),
              ('XHB', 0.0),
              ('XLB', 0.0),
              ('XLE', 0.0),
              ('XLF', 0.0),
              ('XLI

In [53]:
Min_vol_optimized_weights=pd.DataFrame(index=Third_Thursday_Dates, 
                                        columns=Data_prices.columns)

for row in range (len(Min_vol_optimized_weights)):
    for column in range (Min_vol_optimized_weights.shape[1]):
        try:
            column_name=Min_vol_optimized_weights.columns[column]
            row_number=row+1
            Min_vol_optimized_weights.iloc[row,column] =  ETF_min_vol_weights_dict[row_number] [column_name]
        except Exception as e:
            print(f"Error at row {row}, column {column}: {e}")
            
Min_vol_optimized_weights

Ticker,QQQ,SPY,XHB,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XRT,XSD
2015-01-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.75049,0.24951,0.0,0.0,0.0,0.0
2015-02-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.76227,0.23773,0.0,0.0,0.0,0.0
2015-03-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.77236,0.22764,0.0,0.0,0.0,0.0
2015-04-16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.77133,0.22867,0.0,0.0,0.0,0.0
2015-05-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.77135,0.22865,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.65441,0.0,0.34469,0.00091,0.0,0.0
2025-01-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.65492,0.0,0.34473,0.00035,0.0,0.0
2025-02-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.64164,0.0,0.35323,0.00513,0.0,0.0
2025-03-20,0.00916,0.0,0.0,0.0,0.022,0.0,0.0,0.0,0.66847,0.0,0.30037,0.0,0.0,0.0


In [54]:
Min_vol_optimized_weights.to_csv(path_or_buf="Min_vol_optimized_weights.csv")

In [55]:
#The weights will only be realized in the returns tomorrow. Today's return depends on yesterday's weight

Min_vol_daily_weights_df = pd.DataFrame(index = Trading_ETF.index, 
                                        columns = Min_vol_optimized_weights.columns)

# Iterate through rows of Min_vol_daily_weights_df
for row in range(len(Min_vol_daily_weights_df)):
    # Iterate through rows of Min_vol_optimized_weights
    for row_1 in range(len(Min_vol_optimized_weights) - 1):  # Use len(Min_vol_optimized_weights) - 1 to avoid index out of range
        # Compare row names and assign values
        if (
            (Min_vol_daily_weights_df.index[row] >= Min_vol_optimized_weights.index[row_1]) 
            and 
            (Min_vol_daily_weights_df.index[row] < Min_vol_optimized_weights.index[row_1 + 1])
            ):
            Min_vol_daily_weights_df.iloc[row] = Min_vol_optimized_weights.iloc[row_1]
            break  # Exit inner loop after assigning once (assuming you want only one assignment per row)

    # Handle the last row of Min_vol_optimized_weights separately
    last_row_index = len(Min_vol_optimized_weights) - 1
    if Min_vol_daily_weights_df.index[row] >= Min_vol_optimized_weights.index[last_row_index]:
        Min_vol_daily_weights_df.iloc[row] = Min_vol_optimized_weights.iloc[last_row_index]
        
Min_vol_daily_weights_df 

Ticker,QQQ,SPY,XHB,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XRT,XSD
Date,,,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-28,0.0,0.0,0.0,0.0,0.0276,0.0,0.0,0.0,0.57229,0.06684,0.33327,0.0,0.0,0.0
2025-04-29,0.0,0.0,0.0,0.0,0.0276,0.0,0.0,0.0,0.57229,0.06684,0.33327,0.0,0.0,0.0
2025-04-30,0.0,0.0,0.0,0.0,0.0276,0.0,0.0,0.0,0.57229,0.06684,0.33327,0.0,0.0,0.0


### <a id = "p5a">5a.</a>  <font color = "green"> Calculate returns for Min Vol Portfolio Strategy</font>  [back to table of contents](#top)

In [56]:
# Extract the first valid index from Min_vol_daily_weights_df 
first_valid_index = Min_vol_daily_weights_df.first_valid_index()

# Filter Trading_ETF_Daily_Return_df based on this index
Trading_ETF_Daily_Return_df = Trading_ETF_Daily_Return_df[Trading_ETF_Daily_Return_df.index >= first_valid_index]

# Filter Min_vol_daily_weights_df  based on this index
Min_vol_daily_weights_df  = Min_vol_daily_weights_df [Min_vol_daily_weights_df.index >= first_valid_index]

In [57]:
# In this version, Daily_ETF_Portfolio_returns cannot be negative as it is esentially (weights * (1+returns)) and weights cannot be negative

# Initialize a DataFrame for portfolio returns
Daily_Min_Vol_ETF_Portfolio_returns = pd.DataFrame(
    index=Min_vol_daily_weights_df .index, 
    columns=Min_vol_daily_weights_df .columns
)

# Loop through columns and rows to compute portfolio returns
for column in range(Min_vol_daily_weights_df .shape[1]):                           
    for row in range(1, len(Min_vol_daily_weights_df)):
        # Retrieve the row name (date) and column name
        row_name = Min_vol_daily_weights_df .index[row]
        column_name = Min_vol_daily_weights_df .columns[column]

        # Ensure valid data retrieval and handle third Thursday logic
        if Daily_Min_Vol_ETF_Portfolio_returns.index[row - 1] in Third_Thursday_Dates:
            # Apply new weights multiplied by daily returns
            Daily_Min_Vol_ETF_Portfolio_returns.iloc[row, column] = (
                Min_vol_daily_weights_df.iloc[row - 1, column] * 
                (1 + Trading_ETF_Daily_Return_df.loc[row_name, column_name])
            )
        else:
            # Multiply cumulative returns otherwise
            Daily_Min_Vol_ETF_Portfolio_returns.iloc[row, column] = (
                Daily_Min_Vol_ETF_Portfolio_returns.iloc[row - 1, column] * 
                (1 + Trading_ETF_Daily_Return_df.loc[row_name, column_name])
            )

# Calculate the total portfolio return by summing across ETF columns
Daily_Min_Vol_ETF_Portfolio_returns['Total'] = Daily_Min_Vol_ETF_Portfolio_returns.sum(axis=1)

Daily_Min_Vol_ETF_Portfolio_returns

Ticker,QQQ,SPY,XHB,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XRT,XSD,Total
Date,,,,,,,,,,,,,,,
2015-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2015-01-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.74212,0.250223,0.0,0.0,0.0,0.0,0.992343
2015-01-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.74212,0.25007,0.0,0.0,0.0,0.0,0.99219
2015-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7336,0.250427,0.0,0.0,0.0,0.0,0.984026
2015-01-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.72508,0.248288,0.0,0.0,0.0,0.0,0.973368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-28,0.0,0.0,0.0,0.0,0.028098,0.0,0.0,0.0,0.563203,0.067631,0.34092,0.0,0.0,0.0,0.999851
2025-04-29,0.0,0.0,0.0,0.0,0.028017,0.0,0.0,0.0,0.567886,0.068155,0.342592,0.0,0.0,0.0,1.00665
2025-04-30,0.0,0.0,0.0,0.0,0.027261,0.0,0.0,0.0,0.57201,0.067811,0.345519,0.0,0.0,0.0,1.012602


In [58]:
CumulativeLongMinVolStrategyReturnWithoutSL= pd.DataFrame(index=Daily_Min_Vol_ETF_Portfolio_returns.index,
                                                    columns=['Average'])

for row in range(1,len(CumulativeLongMinVolStrategyReturnWithoutSL)):
    if CumulativeLongMinVolStrategyReturnWithoutSL.index[row-1] in Third_Thursday_Dates:                      # If yesterday is Third Thursday
        CumulativeLongMinVolStrategyReturnWithoutSL.iloc[row,:]=Daily_Min_Vol_ETF_Portfolio_returns.iloc[row,-1]-1    # The return series starts anew
    else:
        CumulativeLongMinVolStrategyReturnWithoutSL.iloc[row,:] = \
            (
                (
                    Daily_Min_Vol_ETF_Portfolio_returns.iloc[row,-1]      #If yesterday is not Third Thursday, need to divide to obtain the daily return
                    /
                    Daily_Min_Vol_ETF_Portfolio_returns.iloc[row-1,-1]
                 )-1  
            )
            
CumulativeLongMinVolStrategyReturnWithoutSL

,Average
Date,
2015-01-22,NaN
2015-01-23,-0.007657
2015-01-26,-0.000154
2015-01-27,-0.008228
2015-01-28,-0.010831
...,...
2025-04-28,0.000075
2025-04-29,0.0068
2025-04-30,0.005913


In [59]:
Min_Vol_performance_df, Min_Vol_monthly_statistics=calc_portfolio_statistics(portfolio_df=CumulativeLongMinVolStrategyReturnWithoutSL, 
                                                                    SP500=SP500, 
                                                                    portfolio_name="Min Vol Portfolio, no SL",
                                                                    mean_or_sum="mean")

During this period, Min Vol Portfolio, no SL Cumulative Return was 215.85%
During this period, Min Vol Portfolio, no SL Annualized Return was 7.78%
During this period, Min Vol Portfolio, no SL Annualized Standard Deviation was 14.7%
During this period, Min Vol Portfolio, no SL Sharpe Ratio was 0.53
During this period, Min Vol Portfolio, no SL Sharpe Ratio 95% confidence interval is between 0.53 and 0.53
During this period, Min Vol Portfolio, no SL Sortino Ratio was 0.64
During this period, Min Vol Portfolio, no SL Calmar Ratio was 0.25
During this period, Min Vol Portfolio, no SL Treynor Ratio was 22.22
During this period, Min Vol Portfolio, no SL Max Drawdown was -31.71%
During this period, Min Vol Portfolio, no SL Max Drawdown Duration was 686 days
During this period, Min Vol Portfolio, no SL Average Drawdown was -2.09%
During this period, Min Vol Portfolio, no SL 1 day VaR at the 99% CI was -2.0 %
During this period, Min Vol Portfolio, no SL 1 month VaR at the 99% CI was -11.0 %
Dur

In [60]:
Min_Vol_performance_df

,Average,Returns,CumulativeStrategyReturns,HighWaterMark,Drawdown,MaxDrawdown,HighWaterMarkDate,DrawdownDuration,MaximumDrawdownDuration,DrawdownNumber,OngoingMaxDrawdown,Rolling beta,Month,Year
Date,,,,,,,,,,,,,,
2015-01-23,-0.007657,-0.007657,0.992343,0.992343,0.0,0.0,2015-01-23,0 days,0 days,0,0.000000,NaN,1,2015
2015-01-26,-0.000154,-0.000154,0.99219,0.992343,-0.000154,-0.000154,2015-01-23,3 days,3 days,1,-0.000154,NaN,1,2015
2015-01-27,-0.008228,-0.008228,0.984026,0.992343,-0.008381,-0.008381,2015-01-23,4 days,4 days,1,-0.008381,NaN,1,2015
2015-01-28,-0.010831,-0.010831,0.973368,0.992343,-0.019121,-0.019121,2015-01-23,5 days,5 days,1,-0.019121,NaN,1,2015
2015-01-29,0.009389,0.009389,0.982507,0.992343,-0.009912,-0.019121,2015-01-23,6 days,6 days,1,-0.019121,NaN,1,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-28,0.000075,0.000075,2.142786,2.255925,-0.050152,-0.317086,2024-09-16,224 days,686 days,112,-0.101008,0.346888,4,2025
2025-04-29,0.0068,0.0068,2.157357,2.255925,-0.043693,-0.317086,2024-09-16,225 days,686 days,112,-0.101008,0.347232,4,2025
2025-04-30,0.005913,0.005913,2.170113,2.255925,-0.038038,-0.317086,2024-09-16,226 days,686 days,112,-0.101008,0.348687,4,2025


In [61]:
Min_Vol_monthly_statistics

,Month,Year,Cumulative_Returns,SD,Annualized_Returns,Annualized_SD,Annualized_Sharpe
Date,,,,,,,
2015-01-01,1,2015,-0.036197,0.009716,-0.357518,0.033656,-10.622823
2015-02-01,2,2015,0.014570,0.007199,0.189559,0.024938,7.601075
2015-03-01,3,2015,-0.017093,0.009468,-0.186894,0.032798,-5.698284
2015-04-01,4,2015,-0.006798,0.004839,-0.078598,0.016763,-4.688761
2015-05-01,5,2015,0.008117,0.006023,0.101876,0.020863,4.883029
...,...,...,...,...,...,...,...
2025-01-01,1,2025,0.026697,0.008469,0.371856,0.029339,12.674375
2025-02-01,2,2025,0.038352,0.007266,0.570853,0.025169,22.680753
2025-03-01,3,2025,-0.012291,0.008880,-0.137915,0.030761,-4.483475


In [62]:
plot_chart_with_SP500(portfolio_df = Min_Vol_performance_df['CumulativeStrategyReturns'],
                      chart_title = "Min Vol Strategy Return")

In [63]:
plot_monthly_cumulative_returns(monthly_statistics_df = Min_Vol_monthly_statistics,
                                lookback_period = 24,
                                chart_title = "Last 24 Months of returns of Min Vol Portfolio")

In [64]:
plot_drawdown_chart(portfolio_df = Min_Vol_performance_df, 
                    chart_title = "Min Vol Portfolio Drawdown")

# <a id = "p6">6.</a>  <font color = "green"> Target Return Portfolio</font>  [back to table of contents](#top)

For this section, we set a target return of 7% for the portfolio optimisation.

In [65]:
ETF_target_return_weights_dict = {}

for key,value in ETF_historical_prices.items():
    
    try:

        S = risk_models.CovarianceShrinkage(value).ledoit_wolf()
        
        # You can provide expected returns if you want to
        ef = EfficientFrontier(ETF_expected_returns[key], S, weight_bounds=(0, 1))   # can cap the weights here
        
        # Choose your target
        #ef.min_volatility()
        #ef.max_sharpe(risk_free_rate=historical_risk_free_rate[key])   # Note: For max Sharpe, you probably would not want to provide expected returns 
        ef.efficient_return(target_return=0.07, market_neutral=False)
        #ef.efficient_risk(target_volatility=0.35)
        
        weights = ef.clean_weights()
        
        ETF_target_return_weights_dict[key] = weights
        
    except:
        
        continue

In [66]:
ETF_target_return_weights_dict

{1: OrderedDict([('QQQ', 0.0),
              ('SPY', 0.0),
              ('XHB', 0.0),
              ('XLB', 0.0),
              ('XLE', 0.0),
              ('XLF', 0.0),
              ('XLI', 0.0),
              ('XLK', 0.0),
              ('XLP', 0.75049),
              ('XLU', 0.24951),
              ('XLV', 0.0),
              ('XLY', 0.0),
              ('XRT', 0.0),
              ('XSD', 0.0)]),
 2: OrderedDict([('QQQ', 0.0),
              ('SPY', 0.0),
              ('XHB', 0.0),
              ('XLB', 0.0),
              ('XLE', 0.0),
              ('XLF', 0.0),
              ('XLI', 0.0),
              ('XLK', 0.0),
              ('XLP', 0.76227),
              ('XLU', 0.23773),
              ('XLV', 0.0),
              ('XLY', 0.0),
              ('XRT', 0.0),
              ('XSD', 0.0)]),
 3: OrderedDict([('QQQ', 0.0),
              ('SPY', 0.0),
              ('XHB', 0.0),
              ('XLB', 0.0),
              ('XLE', 0.0),
              ('XLF', 0.0),
              ('XLI

In [67]:
Target_return_optimized_weights=pd.DataFrame(index=Third_Thursday_Dates, 
                                            columns=Data_prices.columns)

for row in range (len(Target_return_optimized_weights)):
    for column in range (Target_return_optimized_weights.shape[1]):
        try:
            column_name = Target_return_optimized_weights.columns[column]
            row_number = row+1
            Target_return_optimized_weights.iloc[row,column] =  ETF_target_return_weights_dict[row_number] [column_name]
        except Exception as e:
            print(f"Error at row {row}, column {column}: {e}")
            Target_return_optimized_weights.iloc[row,column] = 1/Target_return_optimized_weights.shape[1]
            
Target_return_optimized_weights

Error at row 62, column 0: 63
Error at row 62, column 1: 63
Error at row 62, column 2: 63
Error at row 62, column 3: 63
Error at row 62, column 4: 63
Error at row 62, column 5: 63
Error at row 62, column 6: 63
Error at row 62, column 7: 63
Error at row 62, column 8: 63
Error at row 62, column 9: 63
Error at row 62, column 10: 63
Error at row 62, column 11: 63
Error at row 62, column 12: 63
Error at row 62, column 13: 63


Ticker,QQQ,SPY,XHB,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XRT,XSD
2015-01-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.75049,0.24951,0.0,0.0,0.0,0.0
2015-02-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.76227,0.23773,0.0,0.0,0.0,0.0
2015-03-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.77236,0.22764,0.0,0.0,0.0,0.0
2015-04-16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.77133,0.22867,0.0,0.0,0.0,0.0
2015-05-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.77135,0.22865,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.65441,0.0,0.34469,0.00091,0.0,0.0
2025-01-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.65492,0.0,0.34473,0.00035,0.0,0.0
2025-02-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.64164,0.0,0.35323,0.00513,0.0,0.0
2025-03-20,0.00916,0.0,0.0,0.0,0.022,0.0,0.0,0.0,0.66847,0.0,0.30037,0.0,0.0,0.0


In [68]:
Target_return_optimized_weights.to_csv(path_or_buf="Target_return_optimized_weights.csv")

In [69]:
#The weights will only be realized in the returns tomorrow. Today's return depends on yesterday's weight

Target_return_daily_weights_df = pd.DataFrame(index = Trading_ETF.index, 
                                            columns = Target_return_optimized_weights.columns)

# Iterate through rows of Target_return_daily_weights_df
for row in range(len(Target_return_daily_weights_df)):
    # Iterate through rows of Target_return_optimized_weights
    for row_1 in range(len(Target_return_optimized_weights) - 1):  # Use len(Target_return_optimized_weights) - 1 to avoid index out of range
        # Compare row names and assign values
        if (
            (Target_return_daily_weights_df.index[row] >= Target_return_optimized_weights.index[row_1]) 
            and 
            (Target_return_daily_weights_df.index[row] < Target_return_optimized_weights.index[row_1 + 1])
            ):
            Target_return_daily_weights_df.iloc[row] = Target_return_optimized_weights.iloc[row_1]
            break  # Exit inner loop after assigning once (assuming you want only one assignment per row)

    # Handle the last row of Target_return_optimized_weights separately
    last_row_index = len(Target_return_optimized_weights) - 1
    if Target_return_daily_weights_df.index[row] >= Target_return_optimized_weights.index[last_row_index]:
        Target_return_daily_weights_df.iloc[row] = Target_return_optimized_weights.iloc[last_row_index]
        
Target_return_daily_weights_df

Ticker,QQQ,SPY,XHB,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XRT,XSD
Date,,,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-28,0.0,0.0,0.0,0.0,0.0276,0.0,0.0,0.0,0.57229,0.06684,0.33327,0.0,0.0,0.0
2025-04-29,0.0,0.0,0.0,0.0,0.0276,0.0,0.0,0.0,0.57229,0.06684,0.33327,0.0,0.0,0.0
2025-04-30,0.0,0.0,0.0,0.0,0.0276,0.0,0.0,0.0,0.57229,0.06684,0.33327,0.0,0.0,0.0


### <a id = "p6a">6a.</a>  <font color = "green"> Calculate returns for Target Return Portfolio Strategy</font>  [back to table of contents](#top)

In [70]:
# Extract the first valid index from Target_return_daily_weights_df
first_valid_index = Target_return_daily_weights_df.first_valid_index()

# Filter Trading_ETF_Daily_Return_df based on this index
Trading_ETF_Daily_Return_df = Trading_ETF_Daily_Return_df[Trading_ETF_Daily_Return_df.index >= first_valid_index]

# FilterTarget_return_daily_weights_df  based on this index
Target_return_daily_weights_df = Target_return_daily_weights_df [Target_return_daily_weights_df.index >= first_valid_index]

In [71]:
# In this version, Daily_ETF_Portfolio_returns cannot be negative as it is esentially (weights * (1+returns)) and weights cannot be negative

# Initialize a DataFrame for portfolio returns
Daily_Target_Return_ETF_Portfolio_returns = pd.DataFrame(
    index = Target_return_daily_weights_df.index, 
    columns = Target_return_daily_weights_df.columns
)

# Loop through columns and rows to compute portfolio returns
for column in range(Target_return_daily_weights_df.shape[1]):                           
    for row in range(1, len(Target_return_daily_weights_df)):
        # Retrieve the row name (date) and column name
        row_name = Target_return_daily_weights_df.index[row]
        column_name = Target_return_daily_weights_df.columns[column]

        # Ensure valid data retrieval and handle third Thursday logic
        if Daily_Target_Return_ETF_Portfolio_returns.index[row - 1] in Third_Thursday_Dates:
            # Apply new weights multiplied by daily returns
            Daily_Target_Return_ETF_Portfolio_returns.iloc[row, column] = (
                Target_return_daily_weights_df.iloc[row - 1, column] * 
                (1 + Trading_ETF_Daily_Return_df.loc[row_name, column_name])
            )
        else:
            # Multiply cumulative returns otherwise
            Daily_Target_Return_ETF_Portfolio_returns.iloc[row, column] = (
                Daily_Target_Return_ETF_Portfolio_returns.iloc[row - 1, column] * 
                (1 + Trading_ETF_Daily_Return_df.loc[row_name, column_name])
            )

# Calculate the total portfolio return by summing across ETF columns
Daily_Target_Return_ETF_Portfolio_returns['Total'] = Daily_Target_Return_ETF_Portfolio_returns.sum(axis=1)

Daily_Target_Return_ETF_Portfolio_returns

Ticker,QQQ,SPY,XHB,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XRT,XSD,Total
Date,,,,,,,,,,,,,,,
2015-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2015-01-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.74212,0.250223,0.0,0.0,0.0,0.0,0.992343
2015-01-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.74212,0.25007,0.0,0.0,0.0,0.0,0.99219
2015-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7336,0.250427,0.0,0.0,0.0,0.0,0.984026
2015-01-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.72508,0.248288,0.0,0.0,0.0,0.0,0.973368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-28,0.0,0.0,0.0,0.0,0.028098,0.0,0.0,0.0,0.563203,0.067631,0.34092,0.0,0.0,0.0,0.999851
2025-04-29,0.0,0.0,0.0,0.0,0.028017,0.0,0.0,0.0,0.567886,0.068155,0.342592,0.0,0.0,0.0,1.00665
2025-04-30,0.0,0.0,0.0,0.0,0.027261,0.0,0.0,0.0,0.57201,0.067811,0.345519,0.0,0.0,0.0,1.012602


In [72]:
Daily_Target_Return_ETF_Portfolio_returns.to_csv(path_or_buf="Daily_Target_Return_ETF_Portfolio_returns.csv")

In [73]:
CumulativeLongTargetReturnStrategyReturnWithoutSL = pd.DataFrame(index=Daily_Target_Return_ETF_Portfolio_returns.index,
                                                    columns=['Average'])

for row in range(1,len(CumulativeLongTargetReturnStrategyReturnWithoutSL)):
    if CumulativeLongTargetReturnStrategyReturnWithoutSL.index[row-1] in Third_Thursday_Dates:                      # If yesterday is Third Thursday
        CumulativeLongTargetReturnStrategyReturnWithoutSL.iloc[row,:]=Daily_Target_Return_ETF_Portfolio_returns.iloc[row,-1]-1    # The return series starts anew
    else:
        CumulativeLongTargetReturnStrategyReturnWithoutSL.iloc[row,:] = \
            (
                (
                    Daily_Target_Return_ETF_Portfolio_returns.iloc[row,-1]      #If yesterday is not Third Thursday, need to divide to obtain the daily return
                    /
                    Daily_Target_Return_ETF_Portfolio_returns.iloc[row-1,-1]
                 )-1  
            )
            
CumulativeLongTargetReturnStrategyReturnWithoutSL

,Average
Date,
2015-01-22,NaN
2015-01-23,-0.007657
2015-01-26,-0.000154
2015-01-27,-0.008228
2015-01-28,-0.010831
...,...
2025-04-28,0.000075
2025-04-29,0.0068
2025-04-30,0.005913


In [74]:
Target_return_performance_df, Target_return_monthly_statistics = calc_portfolio_statistics(portfolio_df=CumulativeLongTargetReturnStrategyReturnWithoutSL, 
                                                                                        SP500=SP500, 
                                                                                        portfolio_name="Target Return Portfolio, no SL",
                                                                                        mean_or_sum="mean")

During this period, Target Return Portfolio, no SL Cumulative Return was 270.57%
During this period, Target Return Portfolio, no SL Annualized Return was 10.17%
During this period, Target Return Portfolio, no SL Annualized Standard Deviation was 15.68%
During this period, Target Return Portfolio, no SL Sharpe Ratio was 0.65
During this period, Target Return Portfolio, no SL Sharpe Ratio 95% confidence interval is between 0.65 and 0.65
During this period, Target Return Portfolio, no SL Sortino Ratio was 0.81
During this period, Target Return Portfolio, no SL Calmar Ratio was 0.34
During this period, Target Return Portfolio, no SL Treynor Ratio was 29.06
During this period, Target Return Portfolio, no SL Max Drawdown was -30.08%
During this period, Target Return Portfolio, no SL Max Drawdown Duration was 686 days
During this period, Target Return Portfolio, no SL Average Drawdown was -2.03%
During this period, Target Return Portfolio, no SL 1 day VaR at the 99% CI was -3.0 %
During this 

In [75]:
Target_return_performance_df

,Average,Returns,CumulativeStrategyReturns,HighWaterMark,Drawdown,MaxDrawdown,HighWaterMarkDate,DrawdownDuration,MaximumDrawdownDuration,DrawdownNumber,OngoingMaxDrawdown,Rolling beta,Month,Year
Date,,,,,,,,,,,,,,
2015-01-23,-0.007657,-0.007657,0.992343,0.992343,0.0,0.0,2015-01-23,0 days,0 days,0,0.000000,NaN,1,2015
2015-01-26,-0.000154,-0.000154,0.99219,0.992343,-0.000154,-0.000154,2015-01-23,3 days,3 days,1,-0.000154,NaN,1,2015
2015-01-27,-0.008228,-0.008228,0.984026,0.992343,-0.008381,-0.008381,2015-01-23,4 days,4 days,1,-0.008381,NaN,1,2015
2015-01-28,-0.010831,-0.010831,0.973368,0.992343,-0.019121,-0.019121,2015-01-23,5 days,5 days,1,-0.019121,NaN,1,2015
2015-01-29,0.009389,0.009389,0.982507,0.992343,-0.009912,-0.019121,2015-01-23,6 days,6 days,1,-0.019121,NaN,1,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-28,0.000075,0.000075,2.68604,2.827724,-0.050105,-0.300834,2024-09-16,224 days,686 days,129,-0.100964,0.346892,4,2025
2025-04-29,0.0068,0.0068,2.704305,2.827724,-0.043646,-0.300834,2024-09-16,225 days,686 days,129,-0.100964,0.347236,4,2025
2025-04-30,0.005913,0.005913,2.720296,2.827724,-0.037991,-0.300834,2024-09-16,226 days,686 days,129,-0.100964,0.348691,4,2025


In [76]:
Target_return_monthly_statistics

,Month,Year,Cumulative_Returns,SD,Annualized_Returns,Annualized_SD,Annualized_Sharpe
Date,,,,,,,
2015-01-01,1,2015,-0.036197,0.009716,-0.357518,0.033656,-10.622823
2015-02-01,2,2015,0.014570,0.007199,0.189559,0.024938,7.601075
2015-03-01,3,2015,-0.017093,0.009468,-0.186894,0.032798,-5.698284
2015-04-01,4,2015,-0.006798,0.004839,-0.078598,0.016763,-4.688761
2015-05-01,5,2015,0.008117,0.006023,0.101876,0.020863,4.883029
...,...,...,...,...,...,...,...
2025-01-01,1,2025,0.026697,0.008469,0.371856,0.029339,12.674375
2025-02-01,2,2025,0.038352,0.007266,0.570853,0.025169,22.680753
2025-03-01,3,2025,-0.012291,0.008880,-0.137915,0.030761,-4.483475


In [77]:
plot_chart_with_SP500(portfolio_df = Target_return_performance_df['CumulativeStrategyReturns'],
                      chart_title = "Target Return Strategy Return")

In [78]:
plot_monthly_cumulative_returns(monthly_statistics_df = Target_return_monthly_statistics,
                                lookback_period = 24,
                                chart_title = "Last 24 Months of returns of Target Return Portfolio")

In [79]:
plot_drawdown_chart(portfolio_df = Target_return_performance_df, 
                    chart_title = "Target Return Portfolio Drawdown")

# <a id = "p7">7.</a>  <font color = "green"> Summary of strategy results</font>  [back to table of contents](#top)

In [80]:
target_return_stats = output_portfolio_statistics(portfolio_df=CumulativeLongTargetReturnStrategyReturnWithoutSL, 
                                                  SP500=SP500, 
                                                  portfolio_name="Target Return Portfolio, no SL",
                                                  mean_or_sum="mean")

min_vol_stats = output_portfolio_statistics(portfolio_df=CumulativeLongMinVolStrategyReturnWithoutSL, 
                                            SP500=SP500, 
                                            portfolio_name="Min Vol Portfolio, no SL",
                                            mean_or_sum="mean")

tangency_port_stats = output_portfolio_statistics(portfolio_df=CumulativeLongMVOStrategyReturnWithoutSL, 
                                                  SP500=SP500, 
                                                  portfolio_name="MVO, no SL",
                                                  mean_or_sum="mean")

In [81]:
summary_table = pd.DataFrame([tangency_port_stats, min_vol_stats, target_return_stats], index = ["tangency portfolio", "Min vol portfolio", "Target return portfolio"]).T

display(summary_table)

,tangency portfolio,Min vol portfolio,Target return portfolio
Cumulative return (%),318.67,215.85,270.57
Annualized return (%),11.94,7.78,10.17
Annualized Standard Deviation (%),18.66,14.70,15.68
Sharpe ratio,0.64,0.53,0.65
Sortino ratio,0.79,0.64,0.81
Max drawdown (%),-36.55,-31.71,-30.08
Max drawdown duration (days),560.00,686.00,686.00
1 month VaR (%),-15.00,-11.00,-12.00


Comparing the results of the 3 portfolios across several select indicators which would be of interest to portfolio managers, we note several things:

- The tangency portfolio achieved the highest cumulative returns, even vastly outperforming the S&P index over the trade period, but has the highest drawdown. However, the tangency portfolio does not have the highest sharpe ratio, which is the result predicted by Markowitz' modern portfolio theory. This implies that the returns and volatility in the market changes quickly, such that monthly rebalancing is not enough to achieve the desired effect of maximising the reward to risk ratio.

- The minimum vol portfolio indeed has the lowest annualized standard deviation and 1 month VaR, which is an indicator that it has the lowest exposure to market risk among the 3 portfolios. Interestingly, it does not have the lowest maximum drawdown. Given that the same constituent ETFs were used in the construction of the 3 portfolios, it appears that rebalancing based on the past realised vol might not be the best strategy in minimising max drawdown.

- With our choice of target return, the target return portfolio has comparable lower max drawdown than the minimum vol portfolio in terms of max drawdown as a percentage of portfolio size, and duration of drawdown. Its 1 month VaR is also comparable to the minimum vol portfolio. Despite this, it achieved the best Sharpe ratio out of the 3 portfolios. This means that our target return portfolio has captured the essence of both the other portfolios, making the other portfolios unsuitable to be chosen by fund managers seeking to achieve the goals of highest sharpe or lowest vol.